## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [1]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import matplotlib.pyplot as plt
import os
import scipy.signal
import mpld3
mpld3.enable_notebook()


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def BandpassFilter(signal, pass_band):
    """
    Takes the signal and a pass band as inputs

    Returns:
        Filter signal that zeros all frequencies outside the pass band.
    """
    b, a = sp.signal.butter(5, pass_band, btype='bandpass', fs=125)
    return sp.signal.filtfilt(b, a, signal)
    

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def four(sig,fs):
    """
    Takes the signal and a frequency as inputs

    Returns:
        The magnitude of the fourier transform along with the frequencies of the fourier domain
    """
    freqs=np.fft.rfftfreq(len(sig),1/fs)
    fft_mag=np.abs(np.fft.rfft(sig))
    return freqs,fft_mag

def LowpassFilter(signal, fs):
    """
    Takes the signal and a frequency as inputs

    Returns:
     Filter signal that zeros all frequencies higher than the specified frequency.
    """
    b, a = sp.signal.butter(3, 12, btype='lowpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

def Featurize(accx, accy, accz, fs):
    """A partial featurization of the accelerometer signal.
    
    Args:
        accx: (np.array) x-channel of the accelerometer.
        accy: (np.array) y-channel of the accelerometer.
        accz: (np.array) z-channel of the accelerometer.
        fs: (number) the sampling rate of the accelerometer
        
    Returns:
        n-tuple of accelerometer features
    """
    
    accx = LowpassFilter(accx, fs)
    accy = LowpassFilter(accy, fs)
    accz = LowpassFilter(accz, fs)
    
    # The mean of the x-channel
    mn_x = np.mean(accx)

    # The standard deviation of the x-channel
    std_x = np.std(accx)

    # The 5th percentile of the x-channel
    p5_x = np.percentile(accx, 5)

    # The pearson correlation coefficient between the x and y channels
    corr_xy = sp.stats.pearsonr(accx, accy)[0]

    # The total AC energy of the x-axis
    energy_x = np.sum(np.square(accx - np.mean(accx)))  # np.var(accx) * len(accx)
    
    # Take an FFT of the signal. If the signal is too short, 0-pad it so we have at least 2046 points in the FFT.
    fft_len = len(accx)#max(len(accx), 2046)
    
    # Create an array of frequency bins
    freqs = np.fft.rfftfreq(fft_len, 1 / fs)
    
    # Take an FFT of the centered signal
    fft_x = np.fft.rfft(accx - np.mean(accx), fft_len)
    
    # The frequency with the most power between 0.25 and 10 Hz
    low_freqs = (freqs >= 0.25) & (freqs <= 10)
    dominant_frequency_x = freqs[low_freqs][np.argmax(np.abs(fft_x)[low_freqs])]

    # The fraction of energy between 0.25 and 10 Hz in the x-channel
    spectral_energy_x = np.square(np.abs(fft_x))
    energy_23_x = (np.sum(spectral_energy_x[(freqs >= 0.25) & (freqs <= 10)])
                   / np.sum(spectral_energy_x))
    
    return (mn_x,
            std_x,
            p5_x,
            corr_xy,
            energy_x,
            dominant_frequency_x,
            energy_23_x)

def Featurize2(acc, fs):
    """A partial featurization of the accelerometer signal.
    
    Args:
        accx: (np.array) x-channel of the accelerometer.
        accy: (np.array) y-channel of the accelerometer.
        accz: (np.array) z-channel of the accelerometer.
        fs: (number) the sampling rate of the accelerometer
        
    Returns:
        n-tuple of accelerometer features
    """
    
#     acc = LowpassFilter(acc, fs)
    
    # The mean of the x-channel
    mn = np.mean(acc)

    # The standard deviation of the x-channel
    std = np.std(acc)

    # The 5th percentile of the x-channel
    p5 = np.percentile(acc, 5)

#     # The pearson correlation coefficient between the x and y channels
#     corr_xy = sp.stats.pearsonr(accx, accy)[0]

    # The total AC energy of the x-axis
    energy = np.sum(np.square(acc - np.mean(acc)))  # np.var(accx) * len(accx)
    
    # Take an FFT of the signal. If the signal is too short, 0-pad it so we have at least 2046 points in the FFT.
    fft_len = len(acc)#max(len(acc)), 2046)
    
    # Create an array of frequency bins
    freqs = np.fft.rfftfreq(fft_len, 1 / fs)
    
    # Take an FFT of the centered signal
    ffta = np.fft.rfft(acc - np.mean(acc), fft_len)
    
    # The frequency with the most power between 0.25 and 10 Hz
    low_freqs = (freqs >= 0.25) & (freqs <= 10)
    dominant_frequency = freqs[low_freqs][np.argmax(np.abs(ffta)[low_freqs])]

    # The fraction of energy between 0.25 and 10 Hz#### in the x-channel
    spectral_energy = np.square(np.abs(ffta))
    energy_23 = (np.sum(spectral_energy[(freqs >= 0.25) & (freqs <= 10)])
                   / np.sum(spectral_energy))
    
    return (mn,
            std,
            p5,
#             corr_xy,
            energy,
            dominant_frequency,
            energy_23)

# def plot_fft(signal, freqs, fft, fs, plotype):
#     """A function to plot the frequencies of a signal in the fourier domain.
    
#     Args:
#         signal: The original signal.
#         freqs: The frequencies of the signal in the fourier domain.
#         fft: The fourier transform of the signal.
#         fs: (number) the sampling rate frequency of the signal
#         plotype:
        
#     Returns:
#         n-tuple of accelerometer features
#     """
    
#     plt.figure()
#     plt.subplot(2,1,1)
#     ts = np.arange(len(signal)) / fs
#     plt.plot(ts, signal)
#     plt.subplot(2,1,2)
#     plt.plot(freqs, np.abs(fft))
#     plt.title("freq domain of {}".format(plotype))
#     plt.xlabel("freq")
#     plt.tight_layout()
    
def confidencefun(freqs, fft_mag, bpm_f):
    '''
    Function that gets the frequencies and the magnitude of the fourier transform of the signal in a window
    along with an estimated BPM frequency (in Hz) and returns a confidence interval for that frequency
    '''
    window_f = 0.2 #30/60
    fundamental_freq_window = (freqs > bpm_f - window_f) & (freqs < bpm_f + window_f)
    return np.sum(fft_mag[fundamental_freq_window])/ np.sum(fft_mag)

def RunPulseRateAlgorithm(data_fl, ref_fl):#,activ,sub):
    """
    Takes the magnitude of the accelerations in x y and z axis, applies a bandpass filter
    to them and to the PPG signal, and for each window of size 8 sec (next window after 2 sec),
    gets the fourier transform of the magnitude of acceleration and of PPG signal, finds the peak
    values, compares them and if they are different they keep that dominant frequency and if not 
    they calculate the next 4 until it finds the dominant.
    
    Args:
        data_fl: file with the original data.
        ref_fl: file with the ground truth labels.
        
    Returns:
        errors for each dominant frequency along with its confidence interval
    """
    
    fs=125
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl) 

    #Load ground truth labels
    labelsnew = sp.io.loadmat(ref_fl)['BPM0'] #As shown in keys of dictionary
#     print(labelsnew.shape)

    #Get magnitude of accelaration
    magn=np.sqrt(np.square(accx)+np.square(accy)+np.square(accz))

    # Bandpass between 60-180BPM since it was found that minimum BPM is 66 and maximum is 178 in all subjects
    low=60/60
    high=240/60 #had 180/60, changed after feedback
    pass_band=(low,high)
    fil_ppg = BandpassFilter(ppg, pass_band)
    fil_mag = BandpassFilter(magn, pass_band)
#     frequenc,magnit=four(fil_ppg,125)
#     plt.plot(frequenc,magnit)
    
#     #Spectrograms - BPM every 8 sec window and windows overlap every 6 secs
#     #Frequency does not change significantly over time
#     plt.specgram(fil_ppg, Fs=fs) #NFFT=8*fs,noverlap=6*fs,  xextent=((0, 300))     
#     plt.ylim((0, 10))

#     p_spec,p_freq,_,_=plt.specgram(fil_ppg, Fs=fs, NFFT=8*fs, noverlap=6*fs, xextent=((0, 300)))
#     a_spec,a_freq,_,_=plt.specgram(fil_mag, Fs=fs, NFFT=8*fs, noverlap=6*fs, xextent=((0, 300)))
#     print(p_spec.shape)
#     print(p_freq.shape)
#     print(a_spec.shape)
#     print(a_freq.shape)
   
    
    # Compute pulse rate estimates and estimation confidence.
    #Feature extraction
    #Window length is 8 and windows overlap for 6 secs so shift=2sec, based on documentation
    window_length_s = 8
    window_shift_s = 2
    window_length = window_length_s * fs
    window_shift = window_shift_s * fs
    labels, subjects, features = [], [], []
    errors,confidence=[], []
    est_bpm=[]
#     prev_est=0
    ind=-1

    for i in range(0, len(ppg) - window_length, window_shift):
        ind=ind+1
        mag_new=fil_mag[i: i + window_length]
        ppg_new=fil_ppg[i: i + window_length]
        
        #Fourier
        freqs_ac, mags_ac = four(mag_new, fs)
        freqs_p, mags_p = four(ppg_new, fs)
#         plt.plot(freqs_p,mags_p)
        

        
        # Find peaks
        ppg_peaks = sp.signal.find_peaks(mags_p)[0] #, height=2000
#         ppg_peaks_f = freqs_p[ppg_peaks]
        ppg_peaks_f = mags_p[ppg_peaks]
        
#         plt.plot(mags_p)
#         plt.plot(ppg_peaks,mags_p[ppg_peaks],'b.',ms=0.1) #
        
        acc_peaks = sp.signal.find_peaks(mags_ac)[0] #, height=None
#         acc_peaks_f = freqs_ac[acc_peaks]
        acc_peaks_f = mags_ac[acc_peaks]

#         plt.plot(mags_ac)
#         plt.plot(acc_peaks,mags_ac[acc_peaks],'r.',ms=0.1) 


        

        
        ppg_max = freqs_p[np.argmax(mags_p)]
#         print(ppg_max)
#         print("\n")
        acc_max = freqs_ac[np.argmax(mags_ac)]
#         print(acc_max)
#         print(ppg_max-acc_max)

        if (np.abs(ppg_max-acc_max)!=0): #if ac and ppg frequencies are different then this is the pulse for this window
            est_bpm.append(ppg_max*60)
            est_bpm_f=ppg_max
#             print(np.abs(ppg_max-acc_max))
        else: #if these frequencies match then
                  
            k = 1 #find the next 4 candidate frequencies

            while np.abs(acc_max-ppg_max) <= 0.2 and k <=4:
                k+=1
                ppg_max = freqs_p[np.argsort(mags_p, axis=0)[-k]]
                acc_max = freqs_ac[np.argsort(mags_ac, axis=0)[-k]]

            est_bpm_f = ppg_max

#             prev_est = est_bpm_f
            est_bpm.append(est_bpm_f*60)
    
#         est_bpm = [x / 60 for x in est_bpm]
        confidence.append(confidencefun(freqs_p, mags_p, est_bpm_f))
        
        error=np.abs((est_bpm_f*60)-labelsnew[ind])
        error=error.tolist()

        errors.append(error[0])

    return np.asarray(errors), np.asarray(confidence)


In [2]:
# ppg, accx, accy, accz = LoadTroikaDataFile(LoadTroikaDataset()[0][0])
# len(ppg)

In [3]:
err,conf=RunPulseRateAlgorithm(LoadTroikaDataset()[0][0], LoadTroikaDataset()[1][0])
meanabserr=AggregateErrorMetric(err,conf)
meanabserr

30.03125506467154

In [4]:
# features

In [5]:
# labels

In [6]:
Evaluate()

26.870744747297113

In [7]:
# plt.clf()
# plt.title('Spectrogram')
# plt.specgram(ppg, Fs=fs, NFFT=1000, noverlap=75, xextent=((0, T)))
# plt.ylim((0, 20))
# plt.xlabel('Time (sec)')
# plt.ylabel('Frequency (Hz)')

In [8]:
# fs=125
# files=LoadTroikaDataset()
# data=files[0]
# labels=files[1]
# features=[]
# labels2=[]
# subjects=[]
# for i in range(len(data)):
#     if data[i][-5:-4]=='1': #type1 activity
#         activit=1
#     else:
#         activit=2
# #     print(data[i])    
#     feat,lab,subs=RunPulseRateAlgorithm(data[i], labels[i],fs,activ=activit,sub=i)
#     features.append(np.array(feat))
# #     print(np.array(lab))
#     labels2.append(np.array(lab))
#     subs=[x+1 for x in subs]
#     subjects.append(subs) #+1
# features        

In [9]:
# plt.plot(ppg)

In [10]:
# len(features)

In [11]:
# features[0].shape #30 points from 37000+ and 30 datapoints even though 12 subjects since 
#each 10 second window is its own datapoint.

In [12]:
# labels2 = np.array(labels2)
# subjects = np.array(subjects)
# features = np.array(features)

In [13]:
# labels2[0].shape

In [14]:
# subjects

In [15]:
# from sklearn.ensemble import RandomForestClassifier

# n_estimators = 100
# max_tree_depth = 4

# clf = RandomForestClassifier(n_estimators=n_estimators,
#                              max_depth=max_tree_depth,
#                              random_state=42)
# clf.fit(features, labels)

In [16]:
# from sklearn.metrics import confusion_matrix
# from sklearn.model_selection import LeaveOneGroupOut

# class_names = np.array(['class1', 'class2'])
# logo = LeaveOneGroupOut()
# cm = np.zeros((2, 2), dtype='int')

In [17]:
# for train_ind, test_ind in logo.split(features, labels2, subjects):
#     # For each cross-validation fold...
    
#     # Split up the dataset into a training and test set.
#     # The test set has all the data from just one subject
#     X_train, y_train = features[train_ind], labels2[train_ind]
#     X_test, y_test = features[test_ind], labels2[test_ind]
    
#     # Train the classifier
#     clf.fit(X_train, y_train)
    
#     # Run the classifier on the test set
#     y_pred = clf.predict(X_test)
    
#     # Compute the confusion matrix for the test predictions
#     c = confusion_matrix(y_test, y_pred, labels=class_names)
    
#     # Aggregate this confusion matrix with the ones from previous
#     # folds.
#     cm += c

In [18]:
# ppg, accx, accy, accz = LoadTroikaDataFile(LoadTroikaDataset()[0][1])

# fr,ff=four(ppg,125)
# plt.plot(fr,ff)
# plt.title('Frequency-Domain')
# plt.xlabel('Frequency (Hz)')

In [19]:
files=LoadTroikaDataset()
bpmin=[]
bpmax=[]
for i in range(len(files[1])):
    labelsnew=sp.io.loadmat(files[1][i])
    bpmin.append(np.min(labelsnew['BPM0']))
    bpmax.append(np.max(labelsnew['BPM0']))
print("Minimum value of BPM in all subjects is {}".format(np.min(bpmin)))
print("Maximum value of BPM in all subjects is {}".format(np.max(bpmax)))

#     print(np.min(labelsnew['BPM0']))
#     print(np.max(labelsnew['BPM0']))
#     print("\n")
#     print(labelsnew['BPM0'].shape)
    
    #60-180 in total!!!

Minimum value of BPM in all subjects is 66.298
Maximum value of BPM in all subjects is 176.7418


In [20]:
# data_dir = 'datasets/troika/training_data/'

# file=sp.io.loadmat(data_dir+"REF_01_TYPE01.mat")
# file['BPM0'].shape

In [21]:
# data_dir = 'datasets/troika/training_data'
# data_dir2='datasets/troika/training_data/'
# files=os.listdir(data_dir)
# data=[]
# labels=[]
# for i in range(len(files)):
# if 'DATA' in files[i]:
#     file=sp.io.loadmat(data_dir2+files[i])
#     dat=file['sig'][-3:,:]
#     dat=np.transpose(dat)
#     subject=files[i][5:7]
#     activity=files[i][-5:-4]
#     df=pd.DataFrame(data=dat, columns=['accx','accy','accz'])  # 1st row as the column names
#     data.append((subject,activity,df))
# elif 'REF' in files[i]:
#     label=sp.io.loadmat(data_dir2+files[i])
#     labels.append(label['BPM0'])

In [22]:
# import activity_classifier_utils

# data,ground_truth = activity_classifier_utils.LoadWristPPGDataset()
# data

In [23]:
# ground_truth[1].shape

In [24]:
# labels, subjects, features = activity_classifier_utils.GenerateFeatures(data,
#                                                                         fs,
#                                                                         window_length_s=8,
#                                                                         window_shift_s=2)
# labels

In [25]:
# labels.shape

In [26]:
# def LowpassFilter(signal, fs):
#     b, a = sp.signal.butter(3, 12, btype='lowpass', fs=fs)
#     return sp.signal.filtfilt(b, a, signal)

# def Featurize(accx, accy, accz, fs):
#     """A partial featurization of the accelerometer signal.
    
#     Args:
#         accx: (np.array) x-channel of the accelerometer.
#         accy: (np.array) y-channel of the accelerometer.
#         accz: (np.array) z-channel of the accelerometer.
#         fs: (number) the sampling rate of the accelerometer
        
#     Returns:
#         n-tuple of accelerometer features
#     """
    
#     accx = LowpassFilter(accx, fs)
#     accy = LowpassFilter(accy, fs)
#     accz = LowpassFilter(accz, fs)
    
#     # The mean of the x-channel
#     mn_x = np.mean(accx)

#     # The standard deviation of the x-channel
#     std_x = np.std(accx)

#     # The 5th percentile of the x-channel
#     p5_x = np.percentile(accx, 5)

#     # The pearson correlation coefficient between the x and y channels
#     corr_xy = sp.stats.pearsonr(accx, accy)[0]

#     # The total AC energy of the x-axis
#     energy_x = np.sum(np.square(accx - np.mean(accx)))  # np.var(accx) * len(accx)
    
#     # Take an FFT of the signal. If the signal is too short, 0-pad it so we have at least 2046 points in the FFT.
#     fft_len = len(accx)#max(len(accx), 2046)
    
#     # Create an array of frequency bins
#     freqs = np.fft.rfftfreq(fft_len, 1 / fs)
    
#     # Take an FFT of the centered signal
#     fft_x = np.fft.rfft(accx - np.mean(accx), fft_len)
    
#     # The frequency with the most power between 0.25 and 12 Hz
#     low_freqs = (freqs >= 0.25) & (freqs <= 12)
#     dominant_frequency_x = freqs[low_freqs][np.argmax(np.abs(fft_x)[low_freqs])]

#     # The fraction of energy between 2 and 3 Hz in the x-channel
#     spectral_energy_x = np.square(np.abs(fft_x))
#     energy_23_x = (np.sum(spectral_energy_x[(freqs >= 2) & (freqs <= 3)])
#                    / np.sum(spectral_energy_x))
    
#     return (mn_x,
#             std_x,
#             p5_x,
#             corr_xy,
#             energy_x,
#             dominant_frequency_x,
#             energy_23_x)

In [27]:
# import numpy as np
# fs=125
# window_length_s = 8
# window_shift_s = 2 #8 sec -6 overlap
# window_length = window_length_s * fs
# window_shift = window_shift_s * fs
# labels, subjects, features = [], [], []
# for subject, activity, df in data:
#     for i in range(0, len(df) - window_length, window_shift):
#         window = df[i: i + window_length]
#         accx = window.accx.values
#         accy = window.accy.values
#         accz = window.accz.values
#         features.append(activity_classifier_utils.Featurize(accx, accy, accz, fs=fs))
#         labels.append(activity)
#         subjects.append(subject)

In [28]:
# labels = np.array(labels)
# subjects = np.array(subjects)
# features = np.array(features)

In [29]:
# features.shape

In [30]:
# labels.shape

In [31]:
# from sklearn.ensemble import RandomForestClassifier
# n_estimators = 100
# max_tree_depth = 4
# clf = RandomForestClassifier(n_estimators=n_estimators,
#                              max_depth=max_tree_depth,
#                              random_state=42)
# clf.fit(features, ground_truth)

In [32]:
# from sklearn.model_selection import LeaveOneGroupOut
# from sklearn.metrics import confusion_matrix
# class_names = np.array(['01', '02'])
# logo = LeaveOneGroupOut()
# cm = np.zeros((2, 2), dtype='int')
# for train_ind, test_ind in logo.split(features, labels, subjects):
#     # For each cross-validation fold...
    
#     # Split up the dataset into a training and test set.
#     # The test set has all the data from just one subject
#     X_train, y_train = features[train_ind], labels[train_ind]
#     X_test, y_test = features[test_ind], labels[test_ind]
    
#     # Train the classifier
#     clf.fit(X_train, y_train)
    
#     # Run the classifier on the test set
#     y_pred = clf.predict(X_test)
    
#     # Compute the confusion matrix for the test predictions
#     c = confusion_matrix(y_test, y_pred, labels=class_names)
    
#     # Aggregate this confusion matrix with the ones from previous
#     # folds.
#     cm += c

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

> - **Code Description** - By running the 'Evaluate()' function, the pulse rate from a ppg signal and from an accelerometer with x,y and z dimensions will be estimated using the 'RunPulseRateAlgorithm'. It returns an aggregated error on all subjects of troika dataset. The pulse for all subjects ranges from 60 to 180 BPM. More specifically, the 'RunPulseRateAlgorithm' takes the magnitude of the accelerations in x y and z axis, applies a bandpass filter to them and to the PPG signal, and for each window of size 8 sec (next window after 2 sec), gets the fourier transform of the magnitude of acceleration and of PPG signal, finds the peak values, compares them and if they are different they keep that dominant frequency and if not they calculate the next 4 until it finds the dominant. At the end, errors for each dominant frequency along with its confidence interval.
    
> - **Data Description** - Two-channel PPG signals, three-axis acceleration signals, and one-channel ECG signals were
simultaneously recorded from subjects, aged 18-35. All signals were sampled at 125 Hz. Each dataset with the similar name 'DATA_01_TYPE01' contains a variable 'sig'. It has 6 rows. The first row is a simultaneous recording of ECG, which is recorded from the chest of each subject. The second row and the third row are two channels of PPG, which are recorded from the wrist of each subject. The last three rows are simultaneous recordings of acceleration data (in x-, y-, and z-axis). We also provide the calculated ground-truth heart rate, stored in the datasets with the corresponding name, say 'REF_01_TYPE01'. In each of this kind of datasets, there is a variable 'BPM0', which gives the BPM value in every 8-second time window. Note that two successive time windows overlap by 6 seconds. Thus the first value in 'BPM0' gives the calcualted heart rate ground-truth in the first 8 seconds, while the second value in 'BPM0' gives the calculated heart rate ground-truth from the 3rd second to the 10th second. There were 12 subjects in total for this study. Data from different studies should be incorporated if we want to have a more generalizable algorithm.

> - **Algorithhm Description** - The algorithm estimates the BPM for each subject based on accelerometer and PPG data. It first applies a BandPass filter, and then, for each window of 8 seconds (which has an overlap of 6 secs with the previous window) it gets data corresponding to that time, transform them using Fourier, finding the peaks of both accelerometer (square root of sum for each axis) and PPG signal, sort them and compares them to see if they they have different peaks for each dominant frequency. If so, then they consist the pulse estimate. If not, then, for the next 4 dominant frequencies we check the same to find the estimate of the pulse. At the end, a confidence interval and the error of each prediction are estimated. It is worth noticing that algorithm is affected by motion of arms and of the presence of noise. In terms of physiology, when the ventricles contract, the capillaries in the wrist fill with blood. The (typically green) light emitted by the PPG sensor is absorbed by red blood cells in these capillaries and the photodetector will see the drop in reflected light. When the blood returns to the heart, fewer red blood cells in the wrist absorb the light and the photodetector sees an increase in reflected light. The period of this oscillating waveform is the pulse rate.

> - **Algorithm Performance** - Mean Absolute Error (MAE) is used as a performance metric. By running the 'Evaluate()' function we can see that the total error for all subjects is 27.35. For the test subject, the error is 7.28. The algorithm can generalize to data outside of the data used but we should ensure that there is not much noise to them. More specifically, mean absolute error takes the absolute value of the predicted BPM minues the ground truth BPM. Predicted BMP are the estimates that are calculated by running the above algorithm using PPG and accelerometer data. The lower the value the closest the predictions to the ground truth. A mean absolute error of 7.28 for the test subject means that in total, for all the windows of 8 sec (with an overlap of 2 sec) the mean absolute difference of the predictions from the ground truth labels are no more that 7.28 BMP. 

-----
### Next Steps
You will now go to **Test Your Algorithm** (back in the Project Classroom) to apply a unit test to confirm that your algorithm met the success criteria. 